In [41]:
import pandas as pd
from kmodes.kprototypes import KPrototypes
from kmodes.kmodes import KModes
import numpy
import math 

#dados = pd.read_csv('duracao_em_funcao_de_todas_as_variaveis_v2_com_filtro_quartis_sem_tipo_processo.csv', sep=';')
dados = pd.read_csv('duracao_em_funcao_de_todas_as_variaveis_v2_com_filtro_quartis.csv', sep=';')

In [42]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [43]:
import pm4py
from pm4py.objects.log.util import dataframe_utils
from pm4py.objects.conversion.log import converter as log_converter
from pm4py.algo.discovery.dfg import algorithm as dfg_discovery
from pm4py.visualization.dfg import visualizer as dfg_visualization
from pm4py.objects.conversion.dfg import converter as dfg_mining
from pm4py.visualization.petrinet import visualizer as pn_visualizer
from pm4py.algo.filtering.log.variants import variants_filter
from pm4py.statistics.traces.log import case_statistics
from pm4py.statistics.traces.log import case_arrival

In [44]:
#colunas = ["Atuacao_Vara","Median(numero_dias)","Count*(npu)","PIB_percapita","Pop_est_2009"]
colunas = ["Porte_Municipio","Porte_Tribunal","Classificacao_Acervo_2","Atuacao_Vara","Classificacao_PIB_per_capita"]
colunas_str = "Porte_Municipio;Porte_Tribunal;Classificacao_Acervo_2;Atuacao_Vara;Classificacao_PIB_per_capita"
#colunas_num = ["Median(numero_dias)","Count*(npu)","PIB_percapita","Pop_est_2009"]
colunas_num = []
#colunas_cat_nomes = ["tipo_justica","Porte_Municipio","Atuacao_Vara","Classificacao_Acervo_2",]
colunas_cat = [0, 1, 2, 3, 4 ]

#renomear civel pura e criminal pura
dados['Atuacao_Vara'] = dados['Atuacao_Vara'].replace(['Civel Pura'],'Civel')
dados['Atuacao_Vara'] = dados['Atuacao_Vara'].replace(['Criminal Pura'],'Criminal')

#Preenche o porte do tribunal como nao_classifcado
dados['Porte_Tribunal'] = dados['Porte_Tribunal'].fillna('Nao_classificado')


print(dados.head(100))
resultado = pd.DataFrame(columns=['Cluster_config','resultado'])

   codtribunal grau  cod_orgao_julg tipo_justica  \
0         TJAC   G1           11674     Estadual   
1         TJAC   G1           11674     Estadual   
2         TJAC   G1           11674     Estadual   
3         TJAC   G1           14075     Estadual   
4         TJAC   G1           16317     Estadual   
..         ...  ...             ...          ...   
95        TJAC   G2           33563     Estadual   
96        TJAC   G2           74296     Estadual   
97        TJAC   JE           11674     Estadual   
98        TJAC   JE           11674     Estadual   
99        TJAC   JE           16426     Estadual   

                                         descricao  uf Porte_Municipio  \
0                                       VARA CÍVEL  AC   Pequeno_Porte   
1                                       VARA CÍVEL  AC   Pequeno_Porte   
2                                       VARA CÍVEL  AC   Pequeno_Porte   
3   VARA DE EXECUÇÕES PENAS E MEDIDAS ALTERNATIVAS  AC    Grande_Porte   
4    

In [45]:
nome_justica = 'Estadual'
atuacao = 'Fazenda'
dados_segmento = dados
dados_segmento = dados_segmento[dados_segmento['tipo_justica'] == nome_justica]
#dados = dados[dados['codtribunal'] == 'TJPE']
dados_segmento = dados_segmento[dados_segmento['Atuacao_Vara'] == atuacao]
print(dados_segmento.shape)
dados_filter = dados_segmento.filter(items=colunas)
#dados_filter['Median(numero_dias)'] = pd.to_numeric(dados_filter['Median(numero_dias)'])
#dados_filter['PIB_percapita'] = pd.to_numeric(dados_filter['PIB_percapita'])
#dados_filter['Count*(npu)'] = pd.to_numeric(dados_filter['Count*(npu)'])
print(dados_filter.dtypes)
print(dados_filter.head(100))

(222, 16)
Porte_Municipio                 object
Porte_Tribunal                  object
Classificacao_Acervo_2          object
Atuacao_Vara                    object
Classificacao_PIB_per_capita    object
dtype: object
     Porte_Municipio Porte_Tribunal Classificacao_Acervo_2 Atuacao_Vara  \
51      Grande_Porte        PEQUENO          Acervo Grande      Fazenda   
52      Grande_Porte        PEQUENO          Acervo Grande      Fazenda   
53      Grande_Porte        PEQUENO          Acervo Grande      Fazenda   
54      Grande_Porte        PEQUENO          Acervo Grande      Fazenda   
55      Grande_Porte        PEQUENO          Acervo Grande      Fazenda   
...              ...            ...                    ...          ...   
3030    Grande_Porte         GRANDE          Acervo Grande      Fazenda   
3031    Grande_Porte         GRANDE          Acervo Grande      Fazenda   
3032    Grande_Porte         GRANDE          Acervo Grande      Fazenda   
3033    Grande_Porte         GR

In [46]:
for x in range(3,9):
    qtd_clusters = x
    #km = KPrototypes(n_clusters=qtd_clusters, init='Huang', n_init=5, verbose=1)
    km = KModes(n_clusters=qtd_clusters, init='Huang', n_init=5, verbose=1)
    clusters = km.fit_predict(dados_filter, categorical=colunas_cat)
    print('Centroídes')
    print(km.cluster_centroids_)
    print('Clusters')
    print(clusters)
    #dados_cluster = dados_filter
    #dados_cluster['Cluster'] = clusters
    Nome_cluster = 'jus_' + nome_justica + '_atuacao_' + atuacao + '_cluster_' + str(qtd_clusters)
    dados_segmento[Nome_cluster] = clusters
    print(dados_segmento.head(10))
    numpy.savetxt('Clusters_centroides_' + Nome_cluster + '.csv', km.cluster_centroids_ , fmt = '%s', delimiter=";", header=colunas_str, comments='')
    #Agrupamento
    print("Agrupamento...")
    print(str(x))
    dados_agrupados = dados_segmento.filter(items=[Nome_cluster,'Count*(npu)','Sum(numero_dias)']).groupby(by=[Nome_cluster]).sum()
    
    dados_agrupados['media_duracao'] = dados_agrupados['Sum(numero_dias)'] / dados_agrupados['Count*(npu)']
    print(dados_agrupados)
    atual = 0
    anterior = 0
    valor = 0
    media_quad_dif = 0
    for index, row in dados_agrupados.iterrows():
        anterior = atual
        atual = row['media_duracao']
        if anterior > 0:
            valor += pow(anterior-atual, 2)
    media_quad_dif = math.sqrt(valor/qtd_clusters)
    temp = pd.DataFrame({'Cluster_config':[Nome_cluster],'resultado':[media_quad_dif]})
    resultado = resultado.append(temp)
    
#sns.lmplot(data=dados_cluster.filter(items=colunas_num), x='Median(numero_dias)', y='Count*(npu)', hue='Cluster', 
#                   fit_reg=False, legend=True, legend_out=True)

dados_segmento.to_csv('Clusters_jus_' + nome_justica + '_atuacao_' + atuacao +'_classificacoes.csv', sep = ";")
col = ['codtribunal','cod_orgao_julg','descricao','Cluster_3','Cluster_4','Cluster_5','Cluster_6','Cluster_7','Cluster_8']
dados_somente_orgao_julgador_e_cluster = dados_segmento.filter(items=col)
dados_somente_orgao_julgador_e_cluster.to_csv('Clusters_jus_' + nome_justica + '_atuacao_' + atuacao +'_classificacoes_clusters_somente_orgao_julgador_e cluster.csv', sep = ";")

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 34, cost: 155.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 33, cost: 157.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 41, cost: 153.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 45, cost: 144.0
Run 4, iteration: 2/100, moves: 5, cost: 144.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 29, cost: 152.0
Run 5, iteration: 2/100, moves: 4, cost: 152.0
Best run was number 4
Centroídes
[['Medio_Porte' 'GRANDE' 'Acervo Médio' 'Fazenda'
  'Municipio com médio PIB per capita']
 ['Grande_Porte' 'GRANDE' 'Acervo Grande' 'Fazenda'
  'Municipio com alto PIB per capita']
 ['Grande_Porte' 'PEQUENO' 'Acervo Grande' 'Fazenda'

Run 2, iteration: 1/100, moves: 2, cost: 99.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 7, cost: 99.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 0, cost: 111.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 0, cost: 119.0
Best run was number 2
Centroídes
[['Grande_Porte' 'GRANDE' 'Acervo Grande' 'Fazenda'
  'Municipio com alto PIB per capita']
 ['Grande_Porte' 'PEQUENO' 'Acervo Grande' 'Fazenda'
  'Municipio com médio PIB per capita']
 ['Medio_Porte' 'PEQUENO' 'Acervo Grande' 'Fazenda'
  'Municipio com médio PIB per capita']
 ['Medio_Porte' 'GRANDE' 'Acervo Médio' 'Fazenda'
  'Municipio com médio PIB per capita']
 ['Grande_Porte' 'GRANDE' 'Acervo Médio' 'Fazenda'
  'Municipio com alto PIB per capita']]
Clusters
[1 1 1 1 1 1 2 2 2 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 1 1 1 1 1 1 1 1


Starting iterations...
Run 1, iteration: 1/100, moves: 28, cost: 87.0
Run 1, iteration: 2/100, moves: 0, cost: 87.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 13, cost: 88.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 25, cost: 80.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 10, cost: 90.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 30, cost: 81.0
Best run was number 3
Centroídes
[['Grande_Porte' 'GRANDE' 'Acervo Grande' 'Fazenda'
  'Municipio com alto PIB per capita']
 ['Grande_Porte' 'PEQUENO' 'Acervo Grande' 'Fazenda'
  'Municipio com médio PIB per capita']
 ['Grande_Porte' 'GRANDE' 'Acervo Médio' 'Fazenda'
  'Municipio com alto PIB per capita']
 ['Grande_Porte' 'MÉDIO' 'Acervo Grande' 'Fazenda'
  'Municipio com m

Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 52, cost: 70.0
Run 4, iteration: 2/100, moves: 0, cost: 70.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 9, cost: 80.0
Best run was number 1
Centroídes
[['Grande_Porte' 'GRANDE' 'Acervo Médio' 'Fazenda'
  'Municipio com alto PIB per capita']
 ['Grande_Porte' 'GRANDE' 'Acervo Grande' 'Fazenda'
  'Municipio com alto PIB per capita']
 ['Medio_Porte' 'PEQUENO' 'Acervo Grande' 'Fazenda'
  'Municipio com médio PIB per capita']
 ['Grande_Porte' 'PEQUENO' 'Acervo Grande' 'Fazenda'
  'Municipio com médio PIB per capita']
 ['Grande_Porte' 'MÉDIO' 'Acervo Grande' 'Fazenda'
  'Municipio com médio PIB per capita']
 ['Medio_Porte' 'MÉDIO' 'Acervo Médio' 'Fazenda'
  'Municipio com baixo PIB per capita']
 ['Grande_Porte' 'MÉDIO' 'Acervo Grande' 'Fazenda'
  'Municipio com alto PIB per capita']
 ['Medio_Porte' 'GRANDE' 'Acervo Médio' 'Fazenda'
  'Municipi

In [47]:
nome_justica = 'Estadual'
atuacao = 'Execucao fiscal'
dados_segmento = dados
dados_segmento = dados_segmento[dados_segmento['tipo_justica'] == nome_justica]
#dados = dados[dados['codtribunal'] == 'TJPE']
dados_segmento = dados_segmento[dados_segmento['Atuacao_Vara'] == atuacao]
print(dados_segmento.shape)
dados_filter = dados_segmento.filter(items=colunas)
#dados_filter['Median(numero_dias)'] = pd.to_numeric(dados_filter['Median(numero_dias)'])
#dados_filter['PIB_percapita'] = pd.to_numeric(dados_filter['PIB_percapita'])
#dados_filter['Count*(npu)'] = pd.to_numeric(dados_filter['Count*(npu)'])
print(dados_filter.dtypes)
print(dados_filter.head(100))

(119, 16)
Porte_Municipio                 object
Porte_Tribunal                  object
Classificacao_Acervo_2          object
Atuacao_Vara                    object
Classificacao_PIB_per_capita    object
dtype: object
     Porte_Municipio Porte_Tribunal Classificacao_Acervo_2     Atuacao_Vara  \
15      Grande_Porte        PEQUENO          Acervo Grande  Execucao fiscal   
472     Grande_Porte          MÉDIO          Acervo Grande  Execucao fiscal   
475     Grande_Porte          MÉDIO          Acervo Grande  Execucao fiscal   
476     Grande_Porte          MÉDIO          Acervo Grande  Execucao fiscal   
480     Grande_Porte          MÉDIO          Acervo Grande  Execucao fiscal   
...              ...            ...                    ...              ...   
3999     Medio_Porte         GRANDE          Acervo Grande  Execucao fiscal   
4012     Medio_Porte         GRANDE          Acervo Grande  Execucao fiscal   
4025    Grande_Porte         GRANDE          Acervo Grande  Execucao f

In [48]:
for x in range(3,9):
    qtd_clusters = x
    #km = KPrototypes(n_clusters=qtd_clusters, init='Huang', n_init=5, verbose=1)
    km = KModes(n_clusters=qtd_clusters, init='Huang', n_init=5, verbose=1)
    clusters = km.fit_predict(dados_filter, categorical=colunas_cat)
    print('Centroídes')
    print(km.cluster_centroids_)
    print('Clusters')
    print(clusters)
    #dados_cluster = dados_filter
    #dados_cluster['Cluster'] = clusters
    Nome_cluster = 'jus_' + nome_justica + '_atuacao_' + atuacao + '_cluster_' + str(qtd_clusters)
    dados_segmento[Nome_cluster] = clusters
    print(dados_segmento.head(10))
    numpy.savetxt('Clusters_centroides_' + Nome_cluster + '.csv', km.cluster_centroids_ , fmt = '%s', delimiter=";", header=colunas_str, comments='')
    #Agrupamento
    print("Agrupamento...")
    print(str(x))
    dados_agrupados = dados_segmento.filter(items=[Nome_cluster,'Count*(npu)','Sum(numero_dias)']).groupby(by=[Nome_cluster]).sum()
    
    dados_agrupados['media_duracao'] = dados_agrupados['Sum(numero_dias)'] / dados_agrupados['Count*(npu)']
    print(dados_agrupados)
    atual = 0
    anterior = 0
    valor = 0
    media_quad_dif = 0
    for index, row in dados_agrupados.iterrows():
        anterior = atual
        atual = row['media_duracao']
        if anterior > 0:
            valor += pow(anterior-atual, 2)
    media_quad_dif = math.sqrt(valor/qtd_clusters)
    temp = pd.DataFrame({'Cluster_config':[Nome_cluster],'resultado':[media_quad_dif]})
    resultado = resultado.append(temp)
    
#sns.lmplot(data=dados_cluster.filter(items=colunas_num), x='Median(numero_dias)', y='Count*(npu)', hue='Cluster', 
#                   fit_reg=False, legend=True, legend_out=True)

dados_segmento.to_csv('Clusters_jus_' + nome_justica + '_atuacao_' + atuacao +'_classificacoes.csv', sep = ";")
col = ['codtribunal','cod_orgao_julg','descricao','Cluster_3','Cluster_4','Cluster_5','Cluster_6','Cluster_7','Cluster_8']
dados_somente_orgao_julgador_e_cluster = dados_segmento.filter(items=col)
dados_somente_orgao_julgador_e_cluster.to_csv('Clusters_jus_' + nome_justica + '_atuacao_' + atuacao +'_classificacoes_clusters_somente_orgao_julgador_e cluster.csv', sep = ";")

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 10, cost: 60.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 0, cost: 90.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 33, cost: 68.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 24, cost: 60.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 28, cost: 68.0
Best run was number 1
Centroídes
[['Grande_Porte' 'MÉDIO' 'Acervo Grande' 'Execucao fiscal'
  'Municipio com médio PIB per capita']
 ['Grande_Porte' 'MÉDIO' 'Acervo Grande' 'Execucao fiscal'
  'Municipio com alto PIB per capita']
 ['Grande_Porte' 'PEQUENO' 'Acervo Grande' 'Execucao fiscal'
  'Municipio com médio PIB per capita']]
Clusters
[2 1 0 0 0 0 0 0 0 0 0 1 

Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 16, cost: 43.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 19, cost: 52.0
Run 3, iteration: 2/100, moves: 1, cost: 52.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 1, cost: 71.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 0, cost: 83.0
Best run was number 2
Centroídes
[['Medio_Porte' 'MÉDIO' 'Acervo Grande' 'Execucao fiscal'
  'Municipio com médio PIB per capita']
 ['Grande_Porte' 'MÉDIO' 'Acervo Grande' 'Execucao fiscal'
  'Municipio com médio PIB per capita']
 ['Grande_Porte' 'MÉDIO' 'Acervo Grande' 'Execucao fiscal'
  'Municipio com alto PIB per capita']
 ['Grande_Porte' 'MÉDIO' 'Acervo Médio' 'Execucao fiscal'
  'Municipio com alto PIB per capita']
 ['Grande_Porte' 'PEQUENO' 'Acervo Grande' 'Execucao

Starting iterations...
Run 4, iteration: 1/100, moves: 1, cost: 38.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 20, cost: 40.0
Run 5, iteration: 2/100, moves: 0, cost: 40.0
Best run was number 2
Centroídes
[['Grande_Porte' 'GRANDE' 'Acervo Grande' 'Execucao fiscal'
  'Municipio com alto PIB per capita']
 ['Grande_Porte' 'MÉDIO' 'Acervo Grande' 'Execucao fiscal'
  'Municipio com alto PIB per capita']
 ['Grande_Porte' 'PEQUENO' 'Acervo Grande' 'Execucao fiscal'
  'Municipio com alto PIB per capita']
 ['Grande_Porte' 'PEQUENO' 'Acervo Grande' 'Execucao fiscal'
  'Municipio com médio PIB per capita']
 ['Medio_Porte' 'MÉDIO' 'Acervo Grande' 'Execucao fiscal'
  'Municipio com médio PIB per capita']
 ['Grande_Porte' 'MÉDIO' 'Acervo Grande' 'Execucao fiscal'
  'Municipio com médio PIB per capita']]
Clusters
[3 1 5 5 5 5 4 4 5 5 5 1 1 1 5 5 4 4 1 1 1 1 1 1 1 1 1 1 1 4 4 4 4 4 5 5 5
 5 5 5 4 4 4 5 5 5 3 3 3 2 2 2 4 4 1 1 1 1 1 

Starting iterations...
Run 3, iteration: 1/100, moves: 3, cost: 42.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 5, cost: 32.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 5, cost: 25.0
Best run was number 5
Centroídes
[['Grande_Porte' 'MÉDIO' 'Acervo Grande' 'Execucao fiscal'
  'Municipio com médio PIB per capita']
 ['Grande_Porte' 'PEQUENO' 'Acervo Grande' 'Execucao fiscal'
  'Municipio com alto PIB per capita']
 ['Grande_Porte' 'PEQUENO' 'Acervo Grande' 'Execucao fiscal'
  'Municipio com médio PIB per capita']
 ['Medio_Porte' 'MÉDIO' 'Acervo Grande' 'Execucao fiscal'
  'Municipio com médio PIB per capita']
 ['Grande_Porte' 'GRANDE' 'Acervo Grande' 'Execucao fiscal'
  'Municipio com alto PIB per capita']
 ['Grande_Porte' 'MÉDIO' 'Acervo Médio' 'Execucao fiscal'
  'Municipio com alto PIB per capita']
 ['Grande_Porte' 'MÉDIO' 'Acervo Grande' 'Execucao f

Run 4, iteration: 1/100, moves: 3, cost: 21.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 9, cost: 26.0
Run 5, iteration: 2/100, moves: 0, cost: 26.0
Best run was number 3
Centroídes
[['Grande_Porte' 'MÉDIO' 'Acervo Grande' 'Execucao fiscal'
  'Municipio com médio PIB per capita']
 ['Grande_Porte' 'GRANDE' 'Acervo Grande' 'Execucao fiscal'
  'Municipio com médio PIB per capita']
 ['Medio_Porte' 'MÉDIO' 'Acervo Grande' 'Execucao fiscal'
  'Municipio com médio PIB per capita']
 ['Grande_Porte' 'GRANDE' 'Acervo Grande' 'Execucao fiscal'
  'Municipio com alto PIB per capita']
 ['Grande_Porte' 'MÉDIO' 'Acervo Grande' 'Execucao fiscal'
  'Municipio com alto PIB per capita']
 ['Medio_Porte' 'PEQUENO' 'Acervo Grande' 'Execucao fiscal'
  'Municipio com alto PIB per capita']
 ['Grande_Porte' 'PEQUENO' 'Acervo Grande' 'Execucao fiscal'
  'Municipio com alto PIB per capita']
 ['Grande_Porte' 'PEQUENO' 'Acervo Grande' 'Execucao fis

In [49]:
nome_justica = 'Estadual'
atuacao = 'Trabalho'
dados_segmento = dados
dados_segmento = dados_segmento[dados_segmento['tipo_justica'] == nome_justica]
#dados = dados[dados['codtribunal'] == 'TJPE']
dados_segmento = dados_segmento[dados_segmento['Atuacao_Vara'] == atuacao]
print(dados_segmento.shape)
dados_filter = dados_segmento.filter(items=colunas)
#dados_filter['Median(numero_dias)'] = pd.to_numeric(dados_filter['Median(numero_dias)'])
#dados_filter['PIB_percapita'] = pd.to_numeric(dados_filter['PIB_percapita'])
#dados_filter['Count*(npu)'] = pd.to_numeric(dados_filter['Count*(npu)'])
print(dados_filter.dtypes)
print(dados_filter.head(100))

(101, 16)
Porte_Municipio                 object
Porte_Tribunal                  object
Classificacao_Acervo_2          object
Atuacao_Vara                    object
Classificacao_PIB_per_capita    object
dtype: object
     Porte_Municipio Porte_Tribunal Classificacao_Acervo_2 Atuacao_Vara  \
463     Grande_Porte        PEQUENO          Acervo Grande     Trabalho   
1057   Pequeno_Porte          MÉDIO          Acervo Grande     Trabalho   
1058   Pequeno_Porte          MÉDIO          Acervo Grande     Trabalho   
1059   Pequeno_Porte          MÉDIO          Acervo Grande     Trabalho   
1293   Pequeno_Porte          MÉDIO           Acervo Médio     Trabalho   
...              ...            ...                    ...          ...   
2954    Grande_Porte        PEQUENO          Acervo Grande     Trabalho   
2955    Grande_Porte        PEQUENO          Acervo Grande     Trabalho   
2958    Grande_Porte        PEQUENO          Acervo Grande     Trabalho   
2959    Grande_Porte        PEQ

In [50]:
for x in range(3,9):
    qtd_clusters = x
    #km = KPrototypes(n_clusters=qtd_clusters, init='Huang', n_init=5, verbose=1)
    km = KModes(n_clusters=qtd_clusters, init='Huang', n_init=5, verbose=1)
    clusters = km.fit_predict(dados_filter, categorical=colunas_cat)
    print('Centroídes')
    print(km.cluster_centroids_)
    print('Clusters')
    print(clusters)
    #dados_cluster = dados_filter
    #dados_cluster['Cluster'] = clusters
    Nome_cluster = 'jus_' + nome_justica + '_atuacao_' + atuacao + '_cluster_' + str(qtd_clusters)
    dados_segmento[Nome_cluster] = clusters
    print(dados_segmento.head(10))
    numpy.savetxt('Clusters_centroides_' + Nome_cluster + '.csv', km.cluster_centroids_ , fmt = '%s', delimiter=";", header=colunas_str, comments='')
    #Agrupamento
    print("Agrupamento...")
    print(str(x))
    dados_agrupados = dados_segmento.filter(items=[Nome_cluster,'Count*(npu)','Sum(numero_dias)']).groupby(by=[Nome_cluster]).sum()
    
    dados_agrupados['media_duracao'] = dados_agrupados['Sum(numero_dias)'] / dados_agrupados['Count*(npu)']
    print(dados_agrupados)
    atual = 0
    anterior = 0
    valor = 0
    media_quad_dif = 0
    for index, row in dados_agrupados.iterrows():
        anterior = atual
        atual = row['media_duracao']
        if anterior > 0:
            valor += pow(anterior-atual, 2)
    media_quad_dif = math.sqrt(valor/qtd_clusters)
    temp = pd.DataFrame({'Cluster_config':[Nome_cluster],'resultado':[media_quad_dif]})
    resultado = resultado.append(temp)
    
#sns.lmplot(data=dados_cluster.filter(items=colunas_num), x='Median(numero_dias)', y='Count*(npu)', hue='Cluster', 
#                   fit_reg=False, legend=True, legend_out=True)

dados_segmento.to_csv('Clusters_jus_' + nome_justica + '_atuacao_' + atuacao +'_classificacoes.csv', sep = ";")
col = ['codtribunal','cod_orgao_julg','descricao','Cluster_3','Cluster_4','Cluster_5','Cluster_6','Cluster_7','Cluster_8']
dados_somente_orgao_julgador_e_cluster = dados_segmento.filter(items=col)
dados_somente_orgao_julgador_e_cluster.to_csv('Clusters_jus_' + nome_justica + '_atuacao_' + atuacao +'_classificacoes_clusters_somente_orgao_julgador_e cluster.csv', sep = ";")

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 16, cost: 69.0
Run 1, iteration: 2/100, moves: 1, cost: 69.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 49, cost: 77.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 22, cost: 75.0
Run 3, iteration: 2/100, moves: 0, cost: 75.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 0, cost: 90.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 9, cost: 75.0
Best run was number 1
Centroídes
[['Medio_Porte' 'MÉDIO' 'Acervo Médio' 'Trabalho'
  'Municipio com médio PIB per capita']
 ['Pequeno_Porte' 'PEQUENO' 'Acervo Grande' 'Trabalho'
  'Municipio com baixo PIB per capita']
 ['Pequeno_Porte' 'MÉDIO' 'Acervo Grande' 'Trabalho'
  'M

Agrupamento...
4
                                         Count*(npu)  Sum(numero_dias)  \
jus_Estadual_atuacao_Trabalho_cluster_4                                  
0                                                307            445807   
1                                                117            247717   
2                                                190            330665   
3                                                351            355802   

                                         media_duracao  
jus_Estadual_atuacao_Trabalho_cluster_4                 
0                                          1452.140065  
1                                          2117.239316  
2                                          1740.342105  
3                                          1013.680912  
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 22, cost: 49.0
Init: initializing centroids
Init: initializing clusters
Starting ite

Run 3, iteration: 1/100, moves: 4, cost: 49.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 7, cost: 34.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 4, cost: 36.0
Best run was number 4
Centroídes
[['Pequeno_Porte' 'MÉDIO' 'Acervo Grande' 'Trabalho'
  'Municipio com alto PIB per capita']
 ['Pequeno_Porte' 'PEQUENO' 'Acervo Grande' 'Trabalho'
  'Municipio com baixo PIB per capita']
 ['Grande_Porte' 'PEQUENO' 'Acervo Grande' 'Trabalho'
  'Municipio com médio PIB per capita']
 ['Medio_Porte' 'PEQUENO' 'Acervo Grande' 'Trabalho'
  'Municipio com baixo PIB per capita']
 ['Medio_Porte' 'MÉDIO' 'Acervo Médio' 'Trabalho'
  'Municipio com médio PIB per capita']
 ['Pequeno_Porte' 'MÉDIO' 'Acervo Grande' 'Trabalho'
  'Municipio com médio PIB per capita']]
Clusters
[2 0 0 0 4 4 4 4 4 4 0 0 0 4 4 4 0 4 5 5 5 4 4 4 4 4 4 0 0 0 0 0 0 0 0 0 0
 5 5 0 0 4 4 5 0 4 4 4 4 0 

Run 4, iteration: 3/100, moves: 0, cost: 41.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 0, cost: 34.0
Best run was number 5
Centroídes
[['Pequeno_Porte' 'MÉDIO' 'Acervo Grande' 'Trabalho'
  'Municipio com alto PIB per capita']
 ['Grande_Porte' 'PEQUENO' 'Acervo Grande' 'Trabalho'
  'Municipio com médio PIB per capita']
 ['Pequeno_Porte' 'PEQUENO' 'Acervo Grande' 'Trabalho'
  'Municipio com baixo PIB per capita']
 ['Medio_Porte' 'MÉDIO' 'Acervo Médio' 'Trabalho'
  'Municipio com médio PIB per capita']
 ['Medio_Porte' 'PEQUENO' 'Acervo Grande' 'Trabalho'
  'Municipio com baixo PIB per capita']
 ['Pequeno_Porte' 'PEQUENO' 'Acervo Grande' 'Trabalho'
  'Municipio com médio PIB per capita']
 ['Medio_Porte' 'MÉDIO' 'Acervo Grande' 'Trabalho'
  'Municipio com médio PIB per capita']]
Clusters
[1 0 0 0 3 3 3 3 3 3 0 0 0 3 3 3 0 6 0 0 0 3 3 3 3 3 3 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 3 3 0 0 3 3 6 6 0 2 2 2 2 2 2 2 5 5 5 2 2 2 2 2 2 2

Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 8, cost: 39.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 10, cost: 40.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 11, cost: 26.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 11, cost: 22.0
Run 4, iteration: 2/100, moves: 0, cost: 22.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 7, cost: 39.0
Best run was number 4
Centroídes
[['Pequeno_Porte' 'PEQUENO' 'Acervo Grande' 'Trabalho'
  'Municipio com baixo PIB per capita']
 ['Pequeno_Porte' 'MÉDIO' 'Acervo Médio' 'Trabalho'
  'Municipio com médio PIB per capita']
 ['Pequeno_Porte' 'MÉDIO' 'Acervo Médio' 'Trabalho'
  'Municipio com alto PIB per capita']
 ['Medio_Porte' 'PEQUENO' 'Acervo Grand

In [51]:
nome_justica = 'Estadual'
atuacao = 'Juizados civeis'
dados_segmento = dados
dados_segmento = dados_segmento[dados_segmento['tipo_justica'] == nome_justica]
#dados = dados[dados['codtribunal'] == 'TJPE']
dados_segmento = dados_segmento[dados_segmento['Atuacao_Vara'] == atuacao]
print(dados_segmento.shape)
dados_filter = dados_segmento.filter(items=colunas)
#dados_filter['Median(numero_dias)'] = pd.to_numeric(dados_filter['Median(numero_dias)'])
#dados_filter['PIB_percapita'] = pd.to_numeric(dados_filter['PIB_percapita'])
#dados_filter['Count*(npu)'] = pd.to_numeric(dados_filter['Count*(npu)'])
print(dados_filter.dtypes)
print(dados_filter.head(100))

(99, 16)
Porte_Municipio                 object
Porte_Tribunal                  object
Classificacao_Acervo_2          object
Atuacao_Vara                    object
Classificacao_PIB_per_capita    object
dtype: object
     Porte_Municipio Porte_Tribunal Classificacao_Acervo_2     Atuacao_Vara  \
107     Grande_Porte        PEQUENO          Acervo Grande  Juizados civeis   
108     Grande_Porte        PEQUENO          Acervo Grande  Juizados civeis   
242     Grande_Porte        PEQUENO          Acervo Grande  Juizados civeis   
243     Grande_Porte        PEQUENO          Acervo Grande  Juizados civeis   
244     Grande_Porte        PEQUENO          Acervo Grande  Juizados civeis   
...              ...            ...                    ...              ...   
4085    Grande_Porte         GRANDE          Acervo Grande  Juizados civeis   
4090    Grande_Porte         GRANDE          Acervo Grande  Juizados civeis   
4881    Grande_Porte        PEQUENO          Acervo Grande  Juizados ci

In [52]:
for x in range(3,9):
    qtd_clusters = x
    #km = KPrototypes(n_clusters=qtd_clusters, init='Huang', n_init=5, verbose=1)
    km = KModes(n_clusters=qtd_clusters, init='Huang', n_init=5, verbose=1)
    clusters = km.fit_predict(dados_filter, categorical=colunas_cat)
    print('Centroídes')
    print(km.cluster_centroids_)
    print('Clusters')
    print(clusters)
    #dados_cluster = dados_filter
    #dados_cluster['Cluster'] = clusters
    Nome_cluster = 'jus_' + nome_justica + '_atuacao_' + atuacao + '_cluster_' + str(qtd_clusters)
    dados_segmento[Nome_cluster] = clusters
    print(dados_segmento.head(10))
    numpy.savetxt('Clusters_centroides_' + Nome_cluster + '.csv', km.cluster_centroids_ , fmt = '%s', delimiter=";", header=colunas_str, comments='')
    #Agrupamento
    print("Agrupamento...")
    print(str(x))
    dados_agrupados = dados_segmento.filter(items=[Nome_cluster,'Count*(npu)','Sum(numero_dias)']).groupby(by=[Nome_cluster]).sum()
    
    dados_agrupados['media_duracao'] = dados_agrupados['Sum(numero_dias)'] / dados_agrupados['Count*(npu)']
    print(dados_agrupados)
    atual = 0
    anterior = 0
    valor = 0
    media_quad_dif = 0
    for index, row in dados_agrupados.iterrows():
        anterior = atual
        atual = row['media_duracao']
        if anterior > 0:
            valor += pow(anterior-atual, 2)
    media_quad_dif = math.sqrt(valor/qtd_clusters)
    temp = pd.DataFrame({'Cluster_config':[Nome_cluster],'resultado':[media_quad_dif]})
    resultado = resultado.append(temp)
    
#sns.lmplot(data=dados_cluster.filter(items=colunas_num), x='Median(numero_dias)', y='Count*(npu)', hue='Cluster', 
#                   fit_reg=False, legend=True, legend_out=True)

dados_segmento.to_csv('Clusters_jus_' + nome_justica + '_atuacao_' + atuacao +'_classificacoes.csv', sep = ";")
col = ['codtribunal','cod_orgao_julg','descricao','Cluster_3','Cluster_4','Cluster_5','Cluster_6','Cluster_7','Cluster_8']
dados_somente_orgao_julgador_e_cluster = dados_segmento.filter(items=col)
dados_somente_orgao_julgador_e_cluster.to_csv('Clusters_jus_' + nome_justica + '_atuacao_' + atuacao +'_classificacoes_clusters_somente_orgao_julgador_e cluster.csv', sep = ";")

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 26, cost: 68.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 14, cost: 96.0
Run 2, iteration: 2/100, moves: 6, cost: 96.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 25, cost: 74.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 25, cost: 76.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 25, cost: 74.0
Run 5, iteration: 2/100, moves: 28, cost: 68.0
Run 5, iteration: 3/100, moves: 3, cost: 68.0
Best run was number 1
Centroídes
[['Medio_Porte' 'MÉDIO' 'Acervo Grande' 'Juizados civeis'
  'Municipio com médio PIB per capita']
 ['Grande_Porte' 'MÉDIO' 'Acervo Grande' 'Juizados civeis'
  'Municipio com médio PIB per capit

Run 2, iteration: 1/100, moves: 7, cost: 35.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 0, cost: 36.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 2, cost: 73.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 8, cost: 35.0
Run 5, iteration: 2/100, moves: 0, cost: 35.0
Best run was number 2
Centroídes
[['Grande_Porte' 'MÉDIO' 'Acervo Médio' 'Juizados civeis'
  'Municipio com médio PIB per capita']
 ['Medio_Porte' 'MÉDIO' 'Acervo Grande' 'Juizados civeis'
  'Municipio com médio PIB per capita']
 ['Grande_Porte' 'PEQUENO' 'Acervo Grande' 'Juizados civeis'
  'Municipio com médio PIB per capita']
 ['Grande_Porte' 'GRANDE' 'Acervo Grande' 'Juizados civeis'
  'Municipio com alto PIB per capita']
 ['Grande_Porte' 'PEQUENO' 'Acervo Grande' 'Juizados civeis'
  'Municipio com alto PIB per capita']
 

Starting iterations...
Run 5, iteration: 1/100, moves: 15, cost: 34.0
Best run was number 1
Centroídes
[['Medio_Porte' 'MÉDIO' 'Acervo Grande' 'Juizados civeis'
  'Municipio com médio PIB per capita']
 ['Grande_Porte' 'PEQUENO' 'Acervo Grande' 'Juizados civeis'
  'Municipio com médio PIB per capita']
 ['Grande_Porte' 'PEQUENO' 'Acervo Grande' 'Juizados civeis'
  'Municipio com alto PIB per capita']
 ['Grande_Porte' 'MÉDIO' 'Acervo Médio' 'Juizados civeis'
  'Municipio com médio PIB per capita']
 ['Pequeno_Porte' 'PEQUENO' 'Acervo Grande' 'Juizados civeis'
  'Municipio com baixo PIB per capita']
 ['Grande_Porte' 'MÉDIO' 'Acervo Grande' 'Juizados civeis'
  'Municipio com médio PIB per capita']
 ['Pequeno_Porte' 'MÉDIO' 'Acervo Grande' 'Juizados civeis'
  'Municipio com baixo PIB per capita']]
Clusters
[1 1 2 2 2 2 2 2 1 1 1 3 0 0 3 0 0 0 3 3 5 5 5 0 0 5 5 5 5 5 3 5 5 5 5 5 5
 5 0 5 5 5 0 0 0 6 0 0 0 6 5 5 0 2 2 0 0 0 5 5 3 3 3 0 1 1 4 4 4 4 0 0 0 0
 0 0 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 

In [53]:
nome_justica = 'Estadual'
atuacao = 'Familia'
dados_segmento = dados
dados_segmento = dados_segmento[dados_segmento['tipo_justica'] == nome_justica]
#dados = dados[dados['codtribunal'] == 'TJPE']
dados_segmento = dados_segmento[dados_segmento['Atuacao_Vara'] == atuacao]
print(dados_segmento.shape)
dados_filter = dados_segmento.filter(items=colunas)
#dados_filter['Median(numero_dias)'] = pd.to_numeric(dados_filter['Median(numero_dias)'])
#dados_filter['PIB_percapita'] = pd.to_numeric(dados_filter['PIB_percapita'])
#dados_filter['Count*(npu)'] = pd.to_numeric(dados_filter['Count*(npu)'])
print(dados_filter.dtypes)
print(dados_filter.head(100))

(18, 16)
Porte_Municipio                 object
Porte_Tribunal                  object
Classificacao_Acervo_2          object
Atuacao_Vara                    object
Classificacao_PIB_per_capita    object
dtype: object
     Porte_Municipio Porte_Tribunal Classificacao_Acervo_2 Atuacao_Vara  \
2216    Grande_Porte        PEQUENO           Acervo Médio      Familia   
2897    Grande_Porte        PEQUENO          Acervo Grande      Familia   
2898    Grande_Porte        PEQUENO          Acervo Grande      Familia   
2899    Grande_Porte        PEQUENO          Acervo Grande      Familia   
2900    Grande_Porte        PEQUENO           Acervo Médio      Familia   
2901    Grande_Porte        PEQUENO           Acervo Médio      Familia   
2902    Grande_Porte        PEQUENO           Acervo Médio      Familia   
2912    Grande_Porte        PEQUENO          Acervo Grande      Familia   
2913    Grande_Porte        PEQUENO          Acervo Grande      Familia   
2914    Grande_Porte        PEQU

In [54]:
for x in range(3,9):
    qtd_clusters = x
    #km = KPrototypes(n_clusters=qtd_clusters, init='Huang', n_init=5, verbose=1)
    km = KModes(n_clusters=qtd_clusters, init='Huang', n_init=5, verbose=1)
    clusters = km.fit_predict(dados_filter, categorical=colunas_cat)
    print('Centroídes')
    print(km.cluster_centroids_)
    print('Clusters')
    print(clusters)
    #dados_cluster = dados_filter
    #dados_cluster['Cluster'] = clusters
    Nome_cluster = 'jus_' + nome_justica + '_atuacao_' + atuacao + '_cluster_' + str(qtd_clusters)
    dados_segmento[Nome_cluster] = clusters
    print(dados_segmento.head(10))
    numpy.savetxt('Clusters_centroides_' + Nome_cluster + '.csv', km.cluster_centroids_ , fmt = '%s', delimiter=";", header=colunas_str, comments='')
    #Agrupamento
    print("Agrupamento...")
    print(str(x))
    dados_agrupados = dados_segmento.filter(items=[Nome_cluster,'Count*(npu)','Sum(numero_dias)']).groupby(by=[Nome_cluster]).sum()
    
    dados_agrupados['media_duracao'] = dados_agrupados['Sum(numero_dias)'] / dados_agrupados['Count*(npu)']
    print(dados_agrupados)
    atual = 0
    anterior = 0
    valor = 0
    media_quad_dif = 0
    for index, row in dados_agrupados.iterrows():
        anterior = atual
        atual = row['media_duracao']
        if anterior > 0:
            valor += pow(anterior-atual, 2)
    media_quad_dif = math.sqrt(valor/qtd_clusters)
    temp = pd.DataFrame({'Cluster_config':[Nome_cluster],'resultado':[media_quad_dif]})
    resultado = resultado.append(temp)
    
#sns.lmplot(data=dados_cluster.filter(items=colunas_num), x='Median(numero_dias)', y='Count*(npu)', hue='Cluster', 
#                   fit_reg=False, legend=True, legend_out=True)

dados_segmento.to_csv('Clusters_jus_' + nome_justica + '_atuacao_' + atuacao +'_classificacoes.csv', sep = ";")
col = ['codtribunal','cod_orgao_julg','descricao','Cluster_3','Cluster_4','Cluster_5','Cluster_6','Cluster_7','Cluster_8']
dados_somente_orgao_julgador_e_cluster = dados_segmento.filter(items=col)
dados_somente_orgao_julgador_e_cluster.to_csv('Clusters_jus_' + nome_justica + '_atuacao_' + atuacao +'_classificacoes_clusters_somente_orgao_julgador_e cluster.csv', sep = ";")

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/0, moves: 0, cost: 0.0
Centroídes
[['Grande_Porte' 'PEQUENO' 'Acervo Grande' 'Familia'
  'Municipio com médio PIB per capita']
 ['Grande_Porte' 'PEQUENO' 'Acervo Médio' 'Familia'
  'Municipio com médio PIB per capita']]
Clusters
[1 0 0 0 1 1 1 0 0 1 1 1 0 0 0 0 0 0]
     codtribunal grau  cod_orgao_julg tipo_justica  \
2216        TJPB   G1            7439     Estadual   
2897        TJPI   G1           11358     Estadual   
2898        TJPI   G1           11358     Estadual   
2899        TJPI   G1           11358     Estadual   
2900        TJPI   G1           11359     Estadual   
2901        TJPI   G1           11359     Estadual   
2902        TJPI   G1           11359     Estadual   
2912        TJPI   G1           14061     Estadual   
2913        TJPI   G1           14061     Estadual   
2914        TJPI   G1           14062     Estadual   

                                descri

     codtribunal grau  cod_orgao_julg tipo_justica  \
2216        TJPB   G1            7439     Estadual   
2897        TJPI   G1           11358     Estadual   
2898        TJPI   G1           11358     Estadual   
2899        TJPI   G1           11358     Estadual   
2900        TJPI   G1           11359     Estadual   
2901        TJPI   G1           11359     Estadual   
2902        TJPI   G1           11359     Estadual   
2912        TJPI   G1           14061     Estadual   
2913        TJPI   G1           14061     Estadual   
2914        TJPI   G1           14062     Estadual   

                                descricao  uf Porte_Municipio  PIB_percapita  \
2216  ª VARA DE FAMÍLIA DE CAMPINA GRANDE  PB    Grande_Porte   10147.209961   
2897                    ª VARA DA FAMILIA  PI    Grande_Porte   10841.200195   
2898                    ª VARA DA FAMILIA  PI    Grande_Porte   10841.200195   
2899                    ª VARA DA FAMILIA  PI    Grande_Porte   10841.200195   
2900 

In [55]:
nome_justica = 'Eleitoral'
atuacao = 'Eleitoral'
dados_segmento = dados
dados_segmento = dados_segmento[dados_segmento['tipo_justica'] == nome_justica]
#dados = dados[dados['codtribunal'] == 'TJPE']
dados_segmento = dados_segmento[dados_segmento['Atuacao_Vara'] == atuacao]
print(dados_segmento.shape)
dados_filter = dados_segmento.filter(items=colunas)
#dados_filter['Median(numero_dias)'] = pd.to_numeric(dados_filter['Median(numero_dias)'])
#dados_filter['PIB_percapita'] = pd.to_numeric(dados_filter['PIB_percapita'])
#dados_filter['Count*(npu)'] = pd.to_numeric(dados_filter['Count*(npu)'])
print(dados_filter.dtypes)
print(dados_filter.head(100))

(497, 16)
Porte_Municipio                 object
Porte_Tribunal                  object
Classificacao_Acervo_2          object
Atuacao_Vara                    object
Classificacao_PIB_per_capita    object
dtype: object
     Porte_Municipio    Porte_Tribunal Classificacao_Acervo_2 Atuacao_Vara  \
5042    Grande_Porte  Nao_classificado          Acervo Grande    Eleitoral   
5043    Grande_Porte  Nao_classificado          Acervo Grande    Eleitoral   
5044   Pequeno_Porte  Nao_classificado          Acervo Grande    Eleitoral   
5045   Pequeno_Porte  Nao_classificado          Acervo Grande    Eleitoral   
5046   Pequeno_Porte  Nao_classificado          Acervo Grande    Eleitoral   
...              ...               ...                    ...          ...   
5137    Grande_Porte  Nao_classificado          Acervo Grande    Eleitoral   
5138   Pequeno_Porte  Nao_classificado          Acervo Grande    Eleitoral   
5139   Pequeno_Porte  Nao_classificado          Acervo Grande    Eleitoral   
5

In [56]:
for x in range(3,9):
    qtd_clusters = x
    #km = KPrototypes(n_clusters=qtd_clusters, init='Huang', n_init=5, verbose=1)
    km = KModes(n_clusters=qtd_clusters, init='Huang', n_init=5, verbose=1)
    clusters = km.fit_predict(dados_filter, categorical=colunas_cat)
    print('Centroídes')
    print(km.cluster_centroids_)
    print('Clusters')
    print(clusters)
    #dados_cluster = dados_filter
    #dados_cluster['Cluster'] = clusters
    Nome_cluster = 'jus_' + nome_justica + '_atuacao_' + atuacao + '_cluster_' + str(qtd_clusters)
    dados_segmento[Nome_cluster] = clusters
    print(dados_segmento.head(10))
    numpy.savetxt('Clusters_centroides_' + Nome_cluster + '.csv', km.cluster_centroids_ , fmt = '%s', delimiter=";", header=colunas_str, comments='')
    #Agrupamento
    print("Agrupamento...")
    print(str(x))
    dados_agrupados = dados_segmento.filter(items=[Nome_cluster,'Count*(npu)','Sum(numero_dias)']).groupby(by=[Nome_cluster]).sum()
    
    dados_agrupados['media_duracao'] = dados_agrupados['Sum(numero_dias)'] / dados_agrupados['Count*(npu)']
    print(dados_agrupados)
    atual = 0
    anterior = 0
    valor = 0
    media_quad_dif = 0
    for index, row in dados_agrupados.iterrows():
        anterior = atual
        atual = row['media_duracao']
        if anterior > 0:
            valor += pow(anterior-atual, 2)
    media_quad_dif = math.sqrt(valor/qtd_clusters)
    temp = pd.DataFrame({'Cluster_config':[Nome_cluster],'resultado':[media_quad_dif]})
    resultado = resultado.append(temp)
    
#sns.lmplot(data=dados_cluster.filter(items=colunas_num), x='Median(numero_dias)', y='Count*(npu)', hue='Cluster', 
#                   fit_reg=False, legend=True, legend_out=True)

dados_segmento.to_csv('Clusters_jus_' + nome_justica + '_atuacao_' + atuacao +'_classificacoes.csv', sep = ";")
col = ['codtribunal','cod_orgao_julg','descricao','Cluster_3','Cluster_4','Cluster_5','Cluster_6','Cluster_7','Cluster_8']
dados_somente_orgao_julgador_e_cluster = dados_segmento.filter(items=col)
dados_somente_orgao_julgador_e_cluster.to_csv('Clusters_jus_' + nome_justica + '_atuacao_' + atuacao +'_classificacoes_clusters_somente_orgao_julgador_e cluster.csv', sep = ";")

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 138, cost: 421.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 0, cost: 417.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 134, cost: 382.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 108, cost: 425.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 105, cost: 412.0
Best run was number 3
Centroídes
[['Medio_Porte' 'Nao_classificado' 'Acervo Médio' 'Eleitoral'
  'Municipio com médio PIB per capita']
 ['Grande_Porte' 'Nao_classificado' 'Acervo Grande' 'Eleitoral'
  'Municipio com alto PIB per capita']
 ['Pequeno_Porte' 'Nao_classificado' 'Acervo Grande' 'Eleitoral'
  'Municipio com médio PIB per capita']]
Clusters
[1 1

Starting iterations...
Run 1, iteration: 1/100, moves: 69, cost: 305.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 128, cost: 310.0
Run 2, iteration: 2/100, moves: 2, cost: 310.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 122, cost: 267.0
Run 3, iteration: 2/100, moves: 3, cost: 267.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 0, cost: 307.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 0, cost: 294.0
Best run was number 3
Centroídes
[['Grande_Porte' 'Nao_classificado' 'Acervo Grande' 'Eleitoral'
  'Municipio com médio PIB per capita']
 ['Grande_Porte' 'Nao_classificado' 'Acervo Médio' 'Eleitoral'
  'Municipio com médio PIB per capita']
 ['Medio_Porte' 'Nao_classificado' 'Acervo Médio' 'Eleitoral'
  'Municipio com médi

Run 1, iteration: 1/100, moves: 48, cost: 220.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 0, cost: 247.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 60, cost: 289.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 25, cost: 262.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 57, cost: 251.0
Best run was number 1
Centroídes
[['Grande_Porte' 'Nao_classificado' 'Acervo Médio' 'Eleitoral'
  'Municipio com alto PIB per capita']
 ['Grande_Porte' 'Nao_classificado' 'Acervo Grande' 'Eleitoral'
  'Municipio com alto PIB per capita']
 ['Medio_Porte' 'Nao_classificado' 'Acervo Médio' 'Eleitoral'
  'Municipio com médio PIB per capita']
 ['Grande_Porte' 'Nao_classificado' 'Acervo Médio' 'Eleitoral'
  'Municipio com médio PIB per capita'

Run 2, iteration: 1/100, moves: 61, cost: 177.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 83, cost: 242.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 0, cost: 260.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 16, cost: 221.0
Best run was number 1
Centroídes
[['Medio_Porte' 'Nao_classificado' 'Acervo Médio' 'Eleitoral'
  'Municipio com médio PIB per capita']
 ['Grande_Porte' 'Nao_classificado' 'Acervo Médio' 'Eleitoral'
  'Municipio com alto PIB per capita']
 ['Grande_Porte' 'Nao_classificado' 'Acervo Grande' 'Eleitoral'
  'Municipio com alto PIB per capita']
 ['Grande_Porte' 'Nao_classificado' 'Acervo Grande' 'Eleitoral'
  'Municipio com médio PIB per capita']
 ['Pequeno_Porte' 'Nao_classificado' 'Acervo Grande' 'Eleitoral'
  'Municipio com baixo PIB per capita']
 ['Medio_Porte' 'Nao

Run 2, iteration: 1/100, moves: 101, cost: 201.0
Run 2, iteration: 2/100, moves: 46, cost: 201.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 24, cost: 184.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 44, cost: 159.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 36, cost: 160.0
Best run was number 4
Centroídes
[['Grande_Porte' 'Nao_classificado' 'Acervo Grande' 'Eleitoral'
  'Municipio com médio PIB per capita']
 ['Medio_Porte' 'Nao_classificado' 'Acervo Médio' 'Eleitoral'
  'Municipio com médio PIB per capita']
 ['Grande_Porte' 'Nao_classificado' 'Acervo Médio' 'Eleitoral'
  'Municipio com alto PIB per capita']
 ['Grande_Porte' 'Nao_classificado' 'Acervo Grande' 'Eleitoral'
  'Municipio com alto PIB per capita']
 ['Grande_Porte' 'Nao_classificado' 'Acervo Médio' 'Eleitoral'
  'Municipio

In [57]:
nome_justica = 'Militar'
atuacao = 'Militar'
dados_segmento = dados
dados_segmento = dados_segmento[dados_segmento['tipo_justica'] == nome_justica]
#dados = dados[dados['codtribunal'] == 'TJPE']
dados_segmento = dados_segmento[dados_segmento['Atuacao_Vara'] == atuacao]
print(dados_segmento.shape)
dados_filter = dados_segmento.filter(items=colunas)
#dados_filter['Median(numero_dias)'] = pd.to_numeric(dados_filter['Median(numero_dias)'])
#dados_filter['PIB_percapita'] = pd.to_numeric(dados_filter['PIB_percapita'])
#dados_filter['Count*(npu)'] = pd.to_numeric(dados_filter['Count*(npu)'])
print(dados_filter.dtypes)
print(dados_filter.head(100))

(13, 16)
Porte_Municipio                 object
Porte_Tribunal                  object
Classificacao_Acervo_2          object
Atuacao_Vara                    object
Classificacao_PIB_per_capita    object
dtype: object
     Porte_Municipio    Porte_Tribunal Classificacao_Acervo_2 Atuacao_Vara  \
1275    Grande_Porte  Nao_classificado          Acervo Grande      Militar   
1276    Grande_Porte  Nao_classificado          Acervo Grande      Militar   
1277    Grande_Porte  Nao_classificado          Acervo Grande      Militar   
1278    Grande_Porte  Nao_classificado          Acervo Grande      Militar   
1279    Grande_Porte  Nao_classificado          Acervo Grande      Militar   
1280    Grande_Porte  Nao_classificado          Acervo Grande      Militar   
1281    Grande_Porte  Nao_classificado          Acervo Grande      Militar   
1282    Grande_Porte  Nao_classificado          Acervo Grande      Militar   
1283    Grande_Porte  Nao_classificado           Acervo Médio      Militar   
12

In [58]:
for x in range(3,9):
    qtd_clusters = x
    #km = KPrototypes(n_clusters=qtd_clusters, init='Huang', n_init=5, verbose=1)
    km = KModes(n_clusters=qtd_clusters, init='Huang', n_init=5, verbose=1)
    clusters = km.fit_predict(dados_filter, categorical=colunas_cat)
    print('Centroídes')
    print(km.cluster_centroids_)
    print('Clusters')
    print(clusters)
    #dados_cluster = dados_filter
    #dados_cluster['Cluster'] = clusters
    Nome_cluster = 'jus_' + nome_justica + '_atuacao_' + atuacao + '_cluster_' + str(qtd_clusters)
    dados_segmento[Nome_cluster] = clusters
    print(dados_segmento.head(10))
    numpy.savetxt('Clusters_centroides_' + Nome_cluster + '.csv', km.cluster_centroids_ , fmt = '%s', delimiter=";", header=colunas_str, comments='')
    #Agrupamento
    print("Agrupamento...")
    print(str(x))
    dados_agrupados = dados_segmento.filter(items=[Nome_cluster,'Count*(npu)','Sum(numero_dias)']).groupby(by=[Nome_cluster]).sum()
    
    dados_agrupados['media_duracao'] = dados_agrupados['Sum(numero_dias)'] / dados_agrupados['Count*(npu)']
    print(dados_agrupados)
    atual = 0
    anterior = 0
    valor = 0
    media_quad_dif = 0
    for index, row in dados_agrupados.iterrows():
        anterior = atual
        atual = row['media_duracao']
        if anterior > 0:
            valor += pow(anterior-atual, 2)
    media_quad_dif = math.sqrt(valor/qtd_clusters)
    temp = pd.DataFrame({'Cluster_config':[Nome_cluster],'resultado':[media_quad_dif]})
    resultado = resultado.append(temp)
    
#sns.lmplot(data=dados_cluster.filter(items=colunas_num), x='Median(numero_dias)', y='Count*(npu)', hue='Cluster', 
#                   fit_reg=False, legend=True, legend_out=True)

dados_segmento.to_csv('Clusters_jus_' + nome_justica + '_atuacao_' + atuacao +'_classificacoes.csv', sep = ";")
col = ['codtribunal','cod_orgao_julg','descricao','Cluster_3','Cluster_4','Cluster_5','Cluster_6','Cluster_7','Cluster_8']
dados_somente_orgao_julgador_e_cluster = dados_segmento.filter(items=col)
dados_somente_orgao_julgador_e_cluster.to_csv('Clusters_jus_' + nome_justica + '_atuacao_' + atuacao +'_classificacoes_clusters_somente_orgao_julgador_e cluster.csv', sep = ";")

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/0, moves: 0, cost: 0.0
Centroídes
[['Grande_Porte' 'Nao_classificado' 'Acervo Grande' 'Militar'
  'Municipio com alto PIB per capita']
 ['Grande_Porte' 'Nao_classificado' 'Acervo Médio' 'Militar'
  'Municipio com alto PIB per capita']]
Clusters
[0 0 0 0 0 0 0 0 1 1 1 1 1]
     codtribunal grau  cod_orgao_julg tipo_justica  \
1275       TJMSP   G2           29007      Militar   
1276       TJMSP   G2           29007      Militar   
1277       TJMSP   G2           29007      Militar   
1278       TJMSP   G2           29008      Militar   
1279       TJMSP   G2           29008      Militar   
1280       TJMSP   G2           29008      Militar   
1281       TJMSP   G2           29009      Militar   
1282       TJMSP   G2           29009      Militar   
1283       TJMSP   G2           29011      Militar   
1284       TJMSP   G2           29011      Militar   

                                

     codtribunal grau  cod_orgao_julg tipo_justica  \
1275       TJMSP   G2           29007      Militar   
1276       TJMSP   G2           29007      Militar   
1277       TJMSP   G2           29007      Militar   
1278       TJMSP   G2           29008      Militar   
1279       TJMSP   G2           29008      Militar   
1280       TJMSP   G2           29008      Militar   
1281       TJMSP   G2           29009      Militar   
1282       TJMSP   G2           29009      Militar   
1283       TJMSP   G2           29011      Militar   
1284       TJMSP   G2           29011      Militar   

                                      descricao  uf Porte_Municipio  \
1275  GABINETE DO JUIZ AVIVALDI NOGUEIRA JÚNIOR  SP    Grande_Porte   
1276  GABINETE DO JUIZ AVIVALDI NOGUEIRA JÚNIOR  SP    Grande_Porte   
1277  GABINETE DO JUIZ AVIVALDI NOGUEIRA JÚNIOR  SP    Grande_Porte   
1278              GABINETE DO JUIZ PAULO PRAZAK  SP    Grande_Porte   
1279              GABINETE DO JUIZ PAULO PRAZAK  S

In [59]:
nome_justica = 'Trabalho'
atuacao = 'Trabalho'
dados_segmento = dados
dados_segmento = dados_segmento[dados_segmento['tipo_justica'] == nome_justica]
#dados = dados[dados['codtribunal'] == 'TJPE']
dados_segmento = dados_segmento[dados_segmento['Atuacao_Vara'] == atuacao]
print(dados_segmento.shape)
dados_filter = dados_segmento.filter(items=colunas)
#dados_filter['Median(numero_dias)'] = pd.to_numeric(dados_filter['Median(numero_dias)'])
#dados_filter['PIB_percapita'] = pd.to_numeric(dados_filter['PIB_percapita'])
#dados_filter['Count*(npu)'] = pd.to_numeric(dados_filter['Count*(npu)'])
print(dados_filter.dtypes)
print(dados_filter.head(100))

(3565, 16)
Porte_Municipio                 object
Porte_Tribunal                  object
Classificacao_Acervo_2          object
Atuacao_Vara                    object
Classificacao_PIB_per_capita    object
dtype: object
     Porte_Municipio    Porte_Tribunal Classificacao_Acervo_2 Atuacao_Vara  \
6571    Grande_Porte  Nao_classificado          Acervo Grande     Trabalho   
6572    Grande_Porte  Nao_classificado          Acervo Grande     Trabalho   
6573    Grande_Porte  Nao_classificado          Acervo Grande     Trabalho   
6574     Medio_Porte  Nao_classificado           Acervo Médio     Trabalho   
6575     Medio_Porte  Nao_classificado           Acervo Médio     Trabalho   
...              ...               ...                    ...          ...   
6666    Grande_Porte  Nao_classificado          Acervo Grande     Trabalho   
6667    Grande_Porte  Nao_classificado          Acervo Grande     Trabalho   
6668    Grande_Porte  Nao_classificado          Acervo Grande     Trabalho   


In [60]:
for x in range(3,9):
    qtd_clusters = x
    #km = KPrototypes(n_clusters=qtd_clusters, init='Huang', n_init=5, verbose=1)
    km = KModes(n_clusters=qtd_clusters, init='Huang', n_init=5, verbose=1)
    clusters = km.fit_predict(dados_filter, categorical=colunas_cat)
    print('Centroídes')
    print(km.cluster_centroids_)
    print('Clusters')
    print(clusters)
    #dados_cluster = dados_filter
    #dados_cluster['Cluster'] = clusters
    Nome_cluster = 'jus_' + nome_justica + '_atuacao_' + atuacao + '_cluster_' + str(qtd_clusters)
    dados_segmento[Nome_cluster] = clusters
    print(dados_segmento.head(10))
    numpy.savetxt('Clusters_centroides_' + Nome_cluster + '.csv', km.cluster_centroids_ , fmt = '%s', delimiter=";", header=colunas_str, comments='')
    #Agrupamento
    print("Agrupamento...")
    print(str(x))
    dados_agrupados = dados_segmento.filter(items=[Nome_cluster,'Count*(npu)','Sum(numero_dias)']).groupby(by=[Nome_cluster]).sum()
    
    dados_agrupados['media_duracao'] = dados_agrupados['Sum(numero_dias)'] / dados_agrupados['Count*(npu)']
    print(dados_agrupados)
    atual = 0
    anterior = 0
    valor = 0
    media_quad_dif = 0
    for index, row in dados_agrupados.iterrows():
        anterior = atual
        atual = row['media_duracao']
        if anterior > 0:
            valor += pow(anterior-atual, 2)
    media_quad_dif = math.sqrt(valor/qtd_clusters)
    temp = pd.DataFrame({'Cluster_config':[Nome_cluster],'resultado':[media_quad_dif]})
    resultado = resultado.append(temp)
    
#sns.lmplot(data=dados_cluster.filter(items=colunas_num), x='Median(numero_dias)', y='Count*(npu)', hue='Cluster', 
#                   fit_reg=False, legend=True, legend_out=True)

dados_segmento.to_csv('Clusters_jus_' + nome_justica + '_atuacao_' + atuacao +'_classificacoes.csv', sep = ";")
col = ['codtribunal','cod_orgao_julg','descricao','Cluster_3','Cluster_4','Cluster_5','Cluster_6','Cluster_7','Cluster_8']
dados_somente_orgao_julgador_e_cluster = dados_segmento.filter(items=col)
dados_somente_orgao_julgador_e_cluster.to_csv('Clusters_jus_' + nome_justica + '_atuacao_' + atuacao +'_classificacoes_clusters_somente_orgao_julgador_e cluster.csv', sep = ";")

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 45, cost: 862.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 352, cost: 1254.0
Run 2, iteration: 2/100, moves: 323, cost: 1254.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 323, cost: 869.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 89, cost: 869.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 146, cost: 862.0
Best run was number 1
Centroídes
[['Grande_Porte' 'Nao_classificado' 'Acervo Grande' 'Trabalho'
  'Municipio com médio PIB per capita']
 ['Medio_Porte' 'Nao_classificado' 'Acervo Grande' 'Trabalho'
  'Municipio com alto PIB per capita']
 ['Grande_Porte' 'Nao_classificado' 'Acervo Grande' 'Trabalho'
  'Mun

Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 146, cost: 267.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 312, cost: 501.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 127, cost: 501.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 44, cost: 501.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 98, cost: 501.0
Best run was number 1
Centroídes
[['Medio_Porte' 'Nao_classificado' 'Acervo Grande' 'Trabalho'
  'Municipio com alto PIB per capita']
 ['Grande_Porte' 'Nao_classificado' 'Acervo Médio' 'Trabalho'
  'Municipio com alto PIB per capita']
 ['Grande_Porte' 'Nao_classificado' 'Acervo Grande' 'Trabalho'
  'Municipio com médio PIB per capita']
 ['Medio_Porte' 'Nao_classificado' 'Acervo Grand

Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 308, cost: 380.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 5, cost: 607.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 85, cost: 423.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 98, cost: 405.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 52, cost: 173.0
Best run was number 5
Centroídes
[['Medio_Porte' 'Nao_classificado' 'Acervo Grande' 'Trabalho'
  'Municipio com médio PIB per capita']
 ['Medio_Porte' 'Nao_classificado' 'Acervo Médio' 'Trabalho'
  'Municipio com alto PIB per capita']
 ['Grande_Porte' 'Nao_classificado' 'Acervo Grande' 'Trabalho'
  'Municipio com alto PIB per capita']
 ['Grande_Porte' 'Nao_classificado' 'Acervo Grande' 

Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 114, cost: 159.0
Run 1, iteration: 2/100, moves: 0, cost: 159.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 66, cost: 163.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 110, cost: 408.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 48, cost: 198.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 45, cost: 164.0
Best run was number 1
Centroídes
[['Medio_Porte' 'Nao_classificado' 'Acervo Grande' 'Trabalho'
  'Municipio com alto PIB per capita']
 ['Pequeno_Porte' 'Nao_classificado' 'Acervo Grande' 'Trabalho'
  'Municipio com médio PIB per capita']
 ['Grande_Porte' 'Nao_classificado' 'Acervo Grande' 'Trabalho'
  'Municipio com médio PIB per capita']

In [61]:
nome_justica = 'Federal'
atuacao = 'Execucao fiscal'
dados_segmento = dados
dados_segmento = dados_segmento[dados_segmento['tipo_justica'] == nome_justica]
#dados = dados[dados['codtribunal'] == 'TJPE']
dados_segmento = dados_segmento[dados_segmento['Atuacao_Vara'] == atuacao]
print(dados_segmento.shape)
dados_filter = dados_segmento.filter(items=colunas)
#dados_filter['Median(numero_dias)'] = pd.to_numeric(dados_filter['Median(numero_dias)'])
#dados_filter['PIB_percapita'] = pd.to_numeric(dados_filter['PIB_percapita'])
#dados_filter['Count*(npu)'] = pd.to_numeric(dados_filter['Count*(npu)'])
print(dados_filter.dtypes)
print(dados_filter.head(100))

(121, 16)
Porte_Municipio                 object
Porte_Tribunal                  object
Classificacao_Acervo_2          object
Atuacao_Vara                    object
Classificacao_PIB_per_capita    object
dtype: object
     Porte_Municipio    Porte_Tribunal Classificacao_Acervo_2  \
5575    Grande_Porte  Nao_classificado          Acervo Grande   
5576    Grande_Porte  Nao_classificado          Acervo Grande   
5577    Grande_Porte  Nao_classificado          Acervo Grande   
5634    Grande_Porte  Nao_classificado          Acervo Grande   
5635    Grande_Porte  Nao_classificado          Acervo Grande   
...              ...               ...                    ...   
6307    Grande_Porte  Nao_classificado          Acervo Grande   
6308    Grande_Porte  Nao_classificado          Acervo Grande   
6309    Grande_Porte  Nao_classificado          Acervo Grande   
6311    Grande_Porte  Nao_classificado          Acervo Grande   
6312    Grande_Porte  Nao_classificado          Acervo Grande   



In [62]:
for x in range(3,9):
    qtd_clusters = x
    #km = KPrototypes(n_clusters=qtd_clusters, init='Huang', n_init=5, verbose=1)
    km = KModes(n_clusters=qtd_clusters, init='Huang', n_init=5, verbose=1)
    clusters = km.fit_predict(dados_filter, categorical=colunas_cat)
    print('Centroídes')
    print(km.cluster_centroids_)
    print('Clusters')
    print(clusters)
    #dados_cluster = dados_filter
    #dados_cluster['Cluster'] = clusters
    Nome_cluster = 'jus_' + nome_justica + '_atuacao_' + atuacao + '_cluster_' + str(qtd_clusters)
    dados_segmento[Nome_cluster] = clusters
    print(dados_segmento.head(10))
    numpy.savetxt('Clusters_centroides_' + Nome_cluster + '.csv', km.cluster_centroids_ , fmt = '%s', delimiter=";", header=colunas_str, comments='')
    #Agrupamento
    print("Agrupamento...")
    print(str(x))
    dados_agrupados = dados_segmento.filter(items=[Nome_cluster,'Count*(npu)','Sum(numero_dias)']).groupby(by=[Nome_cluster]).sum()
    
    dados_agrupados['media_duracao'] = dados_agrupados['Sum(numero_dias)'] / dados_agrupados['Count*(npu)']
    print(dados_agrupados)
    atual = 0
    anterior = 0
    valor = 0
    media_quad_dif = 0
    for index, row in dados_agrupados.iterrows():
        anterior = atual
        atual = row['media_duracao']
        if anterior > 0:
            valor += pow(anterior-atual, 2)
    media_quad_dif = math.sqrt(valor/qtd_clusters)
    temp = pd.DataFrame({'Cluster_config':[Nome_cluster],'resultado':[media_quad_dif]})
    resultado = resultado.append(temp)
    
#sns.lmplot(data=dados_cluster.filter(items=colunas_num), x='Median(numero_dias)', y='Count*(npu)', hue='Cluster', 
#                   fit_reg=False, legend=True, legend_out=True)

dados_segmento.to_csv('Clusters_jus_' + nome_justica + '_atuacao_' + atuacao +'_classificacoes.csv', sep = ";")
col = ['codtribunal','cod_orgao_julg','descricao','Cluster_3','Cluster_4','Cluster_5','Cluster_6','Cluster_7','Cluster_8']
dados_somente_orgao_julgador_e_cluster = dados_segmento.filter(items=col)
dados_somente_orgao_julgador_e_cluster.to_csv('Clusters_jus_' + nome_justica + '_atuacao_' + atuacao +'_classificacoes_clusters_somente_orgao_julgador_e cluster.csv', sep = ";")

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 0, cost: 26.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 0, cost: 7.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 2, cost: 3.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 1, cost: 7.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 1, cost: 7.0
Best run was number 3
Centroídes
[['Grande_Porte' 'Nao_classificado' 'Acervo Grande' 'Execucao fiscal'
  'Municipio com médio PIB per capita']
 ['Medio_Porte' 'Nao_classificado' 'Acervo Grande' 'Execucao fiscal'
  'Municipio com alto PIB per capita']
 ['Grande_Porte' 'Nao_classificado' 'Acervo Grande' 'Execucao fiscal'
  'Municipio com alto PIB per capita']]
Clusters
[2 2

Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 0, cost: 5.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 1, cost: 1.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 0, cost: 5.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 0, cost: 5.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 0, cost: 1.0
Best run was number 2
Centroídes
[['Grande_Porte' 'Nao_classificado' 'Acervo Grande' 'Execucao fiscal'
  'Municipio com médio PIB per capita']
 ['Grande_Porte' 'Nao_classificado' 'Acervo Médio' 'Execucao fiscal'
  'Municipio com alto PIB per capita']
 ['Medio_Porte' 'Nao_classificado' 'Acervo Grande' 'Execucao fiscal'
  'Municipio com alto PIB per capita']
 ['Grande_Porte' 'Nao_classificado' 'Acervo M

Starting iterations...
Run 1, iteration: 1/0, moves: 0, cost: 0.0
Centroídes
[['Grande_Porte' 'Nao_classificado' 'Acervo Grande' 'Execucao fiscal'
  'Municipio com alto PIB per capita']
 ['Grande_Porte' 'Nao_classificado' 'Acervo Médio' 'Execucao fiscal'
  'Municipio com médio PIB per capita']
 ['Medio_Porte' 'Nao_classificado' 'Acervo Grande' 'Execucao fiscal'
  'Municipio com alto PIB per capita']
 ['Grande_Porte' 'Nao_classificado' 'Acervo Grande' 'Execucao fiscal'
  'Municipio com médio PIB per capita']
 ['Grande_Porte' 'Nao_classificado' 'Acervo Médio' 'Execucao fiscal'
  'Municipio com alto PIB per capita']
 ['Medio_Porte' 'Nao_classificado' 'Acervo Grande' 'Execucao fiscal'
  'Municipio com médio PIB per capita']]
Clusters
[0 0 0 0 3 0 0 0 0 0 0 0 0 0 3 3 3 3 0 0 0 0 0 3 0 0 3 3 0 0 0 0 0 0 0 0 0
 0 5 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 3 3 3
 0 2 3 3 3 0 0 3 3 3 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 4 3
 3 1 3 0 0 0 3 3 3 0]
     codtribu

In [63]:
nome_justica = 'Federal'
atuacao = 'Juizados civeis'
dados_segmento = dados
dados_segmento = dados_segmento[dados_segmento['tipo_justica'] == nome_justica]
#dados = dados[dados['codtribunal'] == 'TJPE']
dados_segmento = dados_segmento[dados_segmento['Atuacao_Vara'] == atuacao]
print(dados_segmento.shape)
dados_filter = dados_segmento.filter(items=colunas)
#dados_filter['Median(numero_dias)'] = pd.to_numeric(dados_filter['Median(numero_dias)'])
#dados_filter['PIB_percapita'] = pd.to_numeric(dados_filter['PIB_percapita'])
#dados_filter['Count*(npu)'] = pd.to_numeric(dados_filter['Count*(npu)'])
print(dados_filter.dtypes)
print(dados_filter.head(100))

(80, 16)
Porte_Municipio                 object
Porte_Tribunal                  object
Classificacao_Acervo_2          object
Atuacao_Vara                    object
Classificacao_PIB_per_capita    object
dtype: object
     Porte_Municipio    Porte_Tribunal Classificacao_Acervo_2  \
5633    Grande_Porte  Nao_classificado           Acervo Médio   
5646    Grande_Porte  Nao_classificado          Acervo Grande   
5647    Grande_Porte  Nao_classificado          Acervo Grande   
5673    Grande_Porte  Nao_classificado          Acervo Grande   
5736    Grande_Porte  Nao_classificado          Acervo Grande   
...              ...               ...                    ...   
6461     Medio_Porte  Nao_classificado           Acervo Médio   
6479    Grande_Porte  Nao_classificado           Acervo Médio   
6480    Grande_Porte  Nao_classificado           Acervo Médio   
6492    Grande_Porte  Nao_classificado           Acervo Médio   
6493    Grande_Porte  Nao_classificado           Acervo Médio   

 

In [64]:
for x in range(3,9):
    qtd_clusters = x
    #km = KPrototypes(n_clusters=qtd_clusters, init='Huang', n_init=5, verbose=1)
    km = KModes(n_clusters=qtd_clusters, init='Huang', n_init=5, verbose=1)
    clusters = km.fit_predict(dados_filter, categorical=colunas_cat)
    print('Centroídes')
    print(km.cluster_centroids_)
    print('Clusters')
    print(clusters)
    #dados_cluster = dados_filter
    #dados_cluster['Cluster'] = clusters
    Nome_cluster = 'jus_' + nome_justica + '_atuacao_' + atuacao + '_cluster_' + str(qtd_clusters)
    dados_segmento[Nome_cluster] = clusters
    print(dados_segmento.head(10))
    numpy.savetxt('Clusters_centroides_' + Nome_cluster + '.csv', km.cluster_centroids_ , fmt = '%s', delimiter=";", header=colunas_str, comments='')
    #Agrupamento
    print("Agrupamento...")
    print(str(x))
    dados_agrupados = dados_segmento.filter(items=[Nome_cluster,'Count*(npu)','Sum(numero_dias)']).groupby(by=[Nome_cluster]).sum()
    
    dados_agrupados['media_duracao'] = dados_agrupados['Sum(numero_dias)'] / dados_agrupados['Count*(npu)']
    print(dados_agrupados)
    atual = 0
    anterior = 0
    valor = 0
    media_quad_dif = 0
    for index, row in dados_agrupados.iterrows():
        anterior = atual
        atual = row['media_duracao']
        if anterior > 0:
            valor += pow(anterior-atual, 2)
    media_quad_dif = math.sqrt(valor/qtd_clusters)
    temp = pd.DataFrame({'Cluster_config':[Nome_cluster],'resultado':[media_quad_dif]})
    resultado = resultado.append(temp)
    
#sns.lmplot(data=dados_cluster.filter(items=colunas_num), x='Median(numero_dias)', y='Count*(npu)', hue='Cluster', 
#                   fit_reg=False, legend=True, legend_out=True)

dados_segmento.to_csv('Clusters_jus_' + nome_justica + '_atuacao_' + atuacao +'_classificacoes.csv', sep = ";")
col = ['codtribunal','cod_orgao_julg','descricao','Cluster_3','Cluster_4','Cluster_5','Cluster_6','Cluster_7','Cluster_8']
dados_somente_orgao_julgador_e_cluster = dados_segmento.filter(items=col)
dados_somente_orgao_julgador_e_cluster.to_csv('Clusters_jus_' + nome_justica + '_atuacao_' + atuacao +'_classificacoes_clusters_somente_orgao_julgador_e cluster.csv', sep = ";")

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 3, cost: 17.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 2, cost: 7.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 5, cost: 29.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 0, cost: 29.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 0, cost: 17.0
Best run was number 2
Centroídes
[['Grande_Porte' 'Nao_classificado' 'Acervo Grande' 'Juizados civeis'
  'Municipio com médio PIB per capita']
 ['Grande_Porte' 'Nao_classificado' 'Acervo Médio' 'Juizados civeis'
  'Municipio com alto PIB per capita']
 ['Grande_Porte' 'Nao_classificado' 'Acervo Grande' 'Juizados civeis'
  'Municipio com alto PIB per capita']]
Clusters
[

Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 0, cost: 12.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 0, cost: 3.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 0, cost: 2.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 0, cost: 12.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 0, cost: 24.0
Best run was number 3
Centroídes
[['Grande_Porte' 'Nao_classificado' 'Acervo Médio' 'Juizados civeis'
  'Municipio com alto PIB per capita']
 ['Medio_Porte' 'Nao_classificado' 'Acervo Grande' 'Juizados civeis'
  'Municipio com alto PIB per capita']
 ['Grande_Porte' 'Nao_classificado' 'Acervo Grande' 'Juizados civeis'
  'Municipio com alto PIB per capita']
 ['Grande_Porte' 'Nao_classificado' 'Acervo

     codtribunal grau  cod_orgao_julg tipo_justica  \
5633        TRF2   G1           12853      Federal   
5646        TRF2   G1           12889      Federal   
5647        TRF2   G1           12890      Federal   
5673        TRF2   G1           17758      Federal   
5736        TRF2   JE           12849      Federal   
5737        TRF2   JE           12850      Federal   
5738        TRF2   JE           12852      Federal   
5739        TRF2   JE           12853      Federal   
5740        TRF2   JE           12860      Federal   
5741        TRF2   JE           12861      Federal   

                                         descricao  uf Porte_Municipio  \
5633  º JUIZADO ESPECIAL FEDERAL DO RIO DE JANEIRO  RJ    Grande_Porte   
5646     º JUIZADO ESPECIAL FEDERAL DE NOVA IGUAÇU  RJ    Grande_Porte   
5647     º JUIZADO ESPECIAL FEDERAL DE NOVA IGUAÇU  RJ    Grande_Porte   
5673            JUIZADO ESPECIAL FEDERAL DE CAMPOS  RJ    Grande_Porte   
5736  º JUIZADO ESPECIAL FEDERAL DO

In [65]:
print(resultado.head(1000))

                                   Cluster_config    resultado
0          jus_Estadual_atuacao_Fazenda_cluster_3   207.415029
0          jus_Estadual_atuacao_Fazenda_cluster_4   701.863170
0          jus_Estadual_atuacao_Fazenda_cluster_5   304.962052
0          jus_Estadual_atuacao_Fazenda_cluster_6   654.345018
0          jus_Estadual_atuacao_Fazenda_cluster_7   403.837310
0          jus_Estadual_atuacao_Fazenda_cluster_8   794.008416
0  jus_Estadual_atuacao_Execucao fiscal_cluster_3   126.858440
0  jus_Estadual_atuacao_Execucao fiscal_cluster_4   168.074011
0  jus_Estadual_atuacao_Execucao fiscal_cluster_5   298.131649
0  jus_Estadual_atuacao_Execucao fiscal_cluster_6   484.153034
0  jus_Estadual_atuacao_Execucao fiscal_cluster_7   620.528157
0  jus_Estadual_atuacao_Execucao fiscal_cluster_8  1052.370290
0         jus_Estadual_atuacao_Trabalho_cluster_3   685.937825
0         jus_Estadual_atuacao_Trabalho_cluster_4   527.362544
0         jus_Estadual_atuacao_Trabalho_cluster_5   800